# FA Group 2

In [1]:
# import libraries

import plotly.graph_objs as go
import pandas
import requests
import textblob
import plotly.express as px
import nltk
from IPython.display import HTML
nltk.download('punkt')
pandas.set_option('display.max_colwidth', -1) # don't cut my pandas dataframes

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lwant\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
<ipython-input-1-f837dd170758>:12: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pandas.set_option('display.max_colwidth', -1) # don't cut my pandas dataframes


In [2]:
from datetime import datetime, timezone

def utc_to_local(utc_dt):
    return utc_dt.replace(tzinfo=timezone.utc).astimezone(tz=None)

In [3]:
# define variables

COMMENT_COLOR         = "blueviolet"
SUBMISSION_COLOR      = "darkorange"
TEXT_PREVIEW_SIZE     = 240
#TERM_OF_INTEREST      = ""
SUBREDDIT_OF_INTEREST = "wallstreetbets"
TIMEFRAME             = "24h" # see more options in the pushshift api docs: https://github.com/pushshift/api\


In [4]:
# a couple of helper functions

def get_pushshift_data(data_type, **kwargs):
    """
    Gets data from the pushshift api.
    
    Data type can be 'comment' or 'submission'
    The rest of the args as interpreted as payload.
    
    Read more: https://github.com/pushshift/api
    """
    
    base_url = f"https://api.pushshift.io/reddit/search/{data_type}/"
    payload = kwargs
    
    request = requests.get(base_url, params=payload)
    
    return request.json()


def make_clickable(val):
    """
    Makes a pandas column clickable.
    """
    
    return '<a href="{}">Link</a>'.format(val)

Ticker

In [5]:
HTML("""\
<style>
.app-subtitle {
    font-size: 1.5em;
}

.app-subtitle a {
    color: #106ba3;
}

.app-subtitle a:hover {
    text-decoration: underline;
}

.app-sidebar p {
    margin-bottom: 1em;
    line-height: 1.7;
}

.app-sidebar a {
    color: #106ba3;
}

.app-sidebar a:hover {
    text-decoration: underline;
}
</style>
""")

In [6]:
df = pandas.read_csv("tickers.csv")

In [7]:
tickers = df.Symbol.tolist()

In [13]:
from ipywidgets import interact, interactive, fixed, interact_manual 
import ipywidgets as widgets

TERM_OF_INTEREST = ''

def f(ticker):
    global TERM_OF_INTEREST
    TERM_OF_INTEREST = ticker
    return ticker

interact(f, ticker=tickers);

In [9]:
# stocks= widgets.Dropdown(
# description='Tickers:   ',
# options=tickers
# )

# def response(name):
#     global TERM_OF_INTEREST
#     TERM_OF_INTEREST = name
    
# display(stocks)

In [10]:
#stocks.observe(response, type='change', names=['value'])

In [14]:
print(TERM_OF_INTEREST)

Dropdown(options=(), value=None)


## Comment activity <a class="anchor" id="1"></a>

In [11]:
data = get_pushshift_data(data_type="comment", q=TERM_OF_INTEREST, after=TIMEFRAME, size=500, aggs="subreddit").get("data")

df = pandas.DataFrame.from_records(data).value_counts("subreddit")[0:10]
x = df.keys()
y = df.values

px.bar(df,
       x=x,
       y=y,
       title=f"Subreddits with most activity - comments with '{TERM_OF_INTEREST}' in the last {TIMEFRAME}",
       labels={"doc_count": "# comments","key": "Subreddits"},
       color_discrete_sequence=[COMMENT_COLOR],
       height=500,
       width=800)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
data = get_pushshift_data(data_type="submission", after=TIMEFRAME, size=500, aggs="created_utc", frequency="hour", sort="desc", subreddit=SUBREDDIT_OF_INTEREST).get("data")
df = pandas.DataFrame.from_records(data).value_counts("created_utc")[0:40]
x = df.keys()
y = df.values

px.scatter(df, x=x, 
               y=y,
               size_max=10,
               labels={"sentiment_polarity": "Comment positivity", "date": "Date comment was posted"},
               title=f"Number of posts in /r/{SUBREDDIT_OF_INTEREST} for the past {TIMEFRAME}",
          )

## Submission activity <a class="anchor" id="2"></a>

In [ ]:
data = get_pushshift_data(data_type="submission", q=TERM_OF_INTEREST, after=TIMEFRAME, frequency="hour", size=500, aggs="subreddit").get("data")

df = pandas.DataFrame.from_records(data).value_counts("subreddit")[0:10]
x = df.keys()
y = df.values


px.bar(df,
       x=x,
       y=y,
       title=f"Subreddits with most activity - submissions with '{TERM_OF_INTEREST}' in the last {TIMEFRAME}",
       labels={"doc_count": "# submissions","key": "Subreddits"},
       color_discrete_sequence=[SUBMISSION_COLOR],
       height=500,
       width=800)

## Most upvoted posts <a class="anchor" id="3"></a>

In [ ]:
data = get_pushshift_data(data_type="submission", q=TERM_OF_INTEREST, after=TIMEFRAME, size=10, sort_type="score", sort="desc").get("data")
# to see what columns interest you, simply list(df)
df = pandas.DataFrame.from_records(data)[["subreddit", "score", "title", "selftext", "permalink"]]

# we only keep the first X characters of the body 
df.title = df.title.str[0:50] + "..."
df.selftext = df.selftext.str[0:50] + "..."

# we append the string to all the permalink entries
df.permalink = "https://reddit.com" + df.permalink.astype(str)

# print 
print(f"\nTop 10 most upvoted posts with '{TERM_OF_INTEREST}' in the past {TIMEFRAME}\n")

# style the last column to be clickable and print
df.style.format({'permalink': make_clickable})

## Most commented posts <a class="anchor" id="4"></a>

In [ ]:
data = get_pushshift_data(data_type="submission", q=TERM_OF_INTEREST, after=TIMEFRAME, size=10, sort_type="num_comments", sort="desc").get("data")


# to see what columns interest you, simply list(df)
df = pandas.DataFrame.from_records(data)[["subreddit", "num_comments", "title", "permalink"]]

# we only keep the first X characters of the body 
df.title = df.title[0:240] + "..."

# we append the string to all the permalink entries
df.permalink = "https://reddit.com" + df.permalink.astype(str)

# print 
print(f"\nTop 10 most commented posts with '{TERM_OF_INTEREST}' in the past {TIMEFRAME}\n")

# style the last column to be clickable and print
df.style.format({'permalink': make_clickable})


## /r/wallstreetbets post title sentiment timeline <a class="anchor" id="5"></a>

In [ ]:
data = get_pushshift_data(q=TERM_OF_INTEREST, data_type="submission", after=TIMEFRAME, size=500, sort_type="score", sort="desc", subreddit=SUBREDDIT_OF_INTEREST).get("data")
df = pandas.DataFrame.from_records(data)[["author", "title", "created_utc", "score", "permalink"]]

df["sentiment_polarity"] = df.apply(lambda row: textblob.TextBlob(row["title"]).sentiment.polarity, axis=1)
df["sentiment_subjectivity"] = df.apply(lambda row: textblob.TextBlob(row["title"]).sentiment.subjectivity, axis=1)
df["sentiment"] = df.apply(lambda row: "positive" if row["sentiment_polarity"] >= 0 else "negative", axis=1)

df["preview"] = df["title"].str[0:50]

df["date"] = pandas.to_datetime(df['created_utc'],unit='s')

px.scatter(df, x="date", 
               y="sentiment_polarity",
               hover_data=["author", "permalink", "preview"],
               color_discrete_map={"positive": "lightseagreen", "negative": "indianred"},
               color="sentiment",
               size_max=10,
               labels={"sentiment_polarity": "Post positivity", "date": "Date post was posted"},
               title=f"Posts' titles containing GME sentiment in /r/{SUBREDDIT_OF_INTEREST} for the past {TIMEFRAME}",
          )


In [ ]:
df1 = df.sentiment.str.split(expand=True).stack().value_counts()
labels = df1.index
values = df1.value_counts().values

fig = px.pie(df, values=df1, names=df1.index, title=f"% of Posts' titles sentiment in /r/{SUBREDDIT_OF_INTEREST} for {TERM_OF_INTEREST} in the past {TIMEFRAME}")
fig.show()

In [ ]:
data = get_pushshift_data(data_type="submission", after=TIMEFRAME, size=500, sort_type="score", sort="desc", subreddit=SUBREDDIT_OF_INTEREST).get("data")
df = pandas.DataFrame.from_records(data)[["title", "selftext", "created_utc","permalink"]]

df["date"] = pandas.to_datetime(df['created_utc'],unit='s')
df
